In [1]:
## required packages
import numpy as np
# import matplotlib.pyplot as plt
# import os, time
import importlib
import im.fcn_im as fcn_im
import pandas as pd
# from numpy.random import standard_normal
# from scipy.linalg import cholesky
import model, fcn_gen, os, time
from scipy import sparse

In [2]:
# base inputs
site_loc_file = r'C:\Users\barry\Desktop\OpenSHAInterface\CA_pipe_midpt_allsites.txt'
rate_min = 1/10000
im_tool = 'sha'

im_dir = r'C:\Users\barry\Desktop\OpenSHAInterface\im_output_10000yr_allsites'
dm_dir = r'C:\Users\barry\Desktop\OpenSHAInterface\rr_out'
rup_meta_file = r'C:\Users\barry\Desktop\OpenSHAInterface\rup_meta_10000yr_allsites.hdf5'
groups2run = np.loadtxt(r'C:\Users\barry\Desktop\OpenSHAInterface\groups2run_allsites.txt',dtype=str)
num_rups = 2
im_list = ['pga','pgv']

## define number of realizations and generate samples
nsamp=10
flag_corr_d = False
flag_corr_T = False

len(groups2run)

1977

In [3]:
##
list_coord_marc = np.asarray([[-121.959102,37.952594],
                  [-121.969138,37.944882],
                  [-121.66069,38.012081],
                  [-121.660565,38.020377],
                  [-121.508865,37.973593],
                  [-121.507891,37.983513],
                  [-121.269766,38.128762],
                  [-121.26935,38.135931]])
ind_marc = [41305, 41312, 29515, 40960, 38581, 76484, 530, 523]
rup_marc = [10, 8, 62]
samp_marc = 5

In [4]:
## reload model.py if it has been modified
importlib.reload(model)
## create class object
A = model.assessment()

In [5]:
count = 0
inc1 = 5
inc2 = 100
# out = {}

In [6]:
## read inputs for Zhu et al. (2017)
zhu_inputs = pd.read_csv('D:\\maps_chris\\CA_Zhu_pipe_midpt_allsites_withWB_remap.csv')
wtd = zhu_inputs['WTD_30m (m)'].values
dr = zhu_inputs['DistRivers (km)'].values
dc = zhu_inputs['DistCoast (km)'].values
dw = zhu_inputs['DistAnyWaterNoWB (km)'].values
precip = zhu_inputs['CA_Precip (mm)'].values
vs30 = zhu_inputs['vs30_opensha (m/s)'].values
dwWB = zhu_inputs['DistAnyWaterWB (km)'].values

## change all -9999 entries to NaN
find_val = -9999
set_val = np.nan
wtd = fcn_gen.find_set_nan(wtd,find_val,set_val)
dr = fcn_gen.find_set_nan(dr,find_val,set_val)
dc = fcn_gen.find_set_nan(dc,find_val,set_val)
dw = fcn_gen.find_set_nan(dw,find_val,set_val)
precip = fcn_gen.find_set_nan(precip,find_val,set_val)
vs30 = fcn_gen.find_set_nan(vs30,find_val,set_val)
dwWB = fcn_gen.find_set_nan(dwWB,find_val,set_val)

## elevation from DEM maps
z = np.ones(vs30.shape)*10 ## set to 10 for now, get data from DEM map later

In [7]:
## Lateral spreading
time_start = time.time()

multi = 0
range_start = 0+100*multi
range_end = range_start+1
flag_sample_exist = True

##
for im_group in groups2run[range_start:range_end]:
    #####################################################################
    count += 1
    path_sample = im_dir+'/'+im_group+'/'
    
    #####################################################################
    ## load GM predictions and create random variable
    A.create_RV(im_tool, site_loc_file, im_dir, im_list, rup_meta_file, flag_clear_dict=True, 
                flag_sample_exist=flag_sample_exist, im_group=im_group, num_rups=num_rups)

    #####################################################################
    ## generate/import samples
    A.get_IM(nsamp, im_list, flag_corr_d, flag_corr_T, flag_clear_dict=True, 
            flag_sample_exist=flag_sample_exist,path_sample=path_sample)

    if flag_sample_exist is False:
        for i in range(nsamp):
            out1 = sparse.csc_matrix(np.round(A._IM_dict['pgv'][i].log1p(),decimals=3))
            out2 = sparse.csc_matrix(np.round(A._IM_dict['pga'][i].log1p(),decimals=3))
            sparse.save_npz(im_dir+'/'+im_group+'/'+'pgv_samp_'+str(i)+'.npz',out1)
            sparse.save_npz(im_dir+'/'+im_group+'/'+'pga_samp_'+str(i)+'.npz',out2)

    #####################################################################
    ## get liq_susc from Zhu et al. (2017)
#     category = 'liq'
#     method = 'zhu_etal_2017'
#     return_param = ['liq_susc']
#     dc_cutoff = 20
#     A.get_EDP(category=category, method=method, return_param=return_param, 
#               vs30=vs30, precip=precip, dc=dc, dr=dr, dw=dw, 
#               wtd=wtd, dc_cutoff=dc_cutoff)
    
    ## get p_liq and liq_susc from Zhu et al. (2017)
    category = 'liq'
    method = 'zhu_etal_2017'
    return_param = ['liq_susc','p_liq']
    flag_pgv = True
    flag_M = True
    dc_cutoff = 20
    A.get_EDP(category=category, method=method, return_param=return_param,
              flag_pgv=flag_pgv, flag_M=flag_M, vs30=vs30, precip=precip, dc=dc, dr=dr, dw=dw, 
              wtd=wtd, dc_cutoff=dc_cutoff, nsamp=nsamp)
    
    ## get pgd_ls from Grant et al. (2016), same as HAZUS (FEMA, 2014)
    category = 'ls'
    method = 'hazus_2014_ls'
    return_param = ['pgd_ls']
    source_dict = ['_EDP_dict']
    source_param = ['liq_susc']
    source_method = ['zhu_etal_2017']
    flag_pga = True
    flag_M = True
    dw_cutoff = 50
    store_name = ['pgd_ls_dw'+str(dw_cutoff)]
    A.get_EDP(category=category, method=method, return_param=return_param,
              flag_pga=flag_pga, flag_M=flag_M, source_dict=source_dict, 
              source_param=source_param, source_method=source_method, dw=dwWB*1000, z=z, nsamp=nsamp,
              flag_extrap_Epgd=True, dw_cutoff=dw_cutoff)

    #####################################################################
    dm_method = []
    ## get rr from HAZUS (FEMA, 2014)
    category = 'rr'
    method = 'hazus_2014_rr'
    return_param = ['rr_pgd']
    source_dict = ['_EDP_dict','_EDP_dict']
    source_param = ['p_liq','pgd_ls']
    source_method = ['zhu_etal_2017','hazus_2014_ls']
    pgd_label = 'pgd_ls'
    flag_rup_depend = True
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category=category, method=method, return_param=return_param, flag_rup_depend=flag_rup_depend,
              source_dict=source_dict, source_param=source_param, source_method=source_method, 
              nsamp=nsamp, pgd_label=pgd_label)
    
    ## get rr from ALA (2001)
    category = 'rr'
    method = 'orourke_2020_rr'
    return_param = ['rr_pgd']
    source_dict = ['_EDP_dict']
    source_param = ['pgd_ls']
    source_method = ['hazus_2014_ls']
    pgd_label = 'pgd_ls'
    flag_rup_depend = True
    pgd_cutoff = 0
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category=category, method=method, return_param=return_param, flag_rup_depend=flag_rup_depend,
                source_dict=source_dict, source_param=source_param, source_method=source_method, nsamp=nsamp,
                pgd_cutoff=pgd_cutoff, pgd_label=pgd_label)

    #####################################################################
#     if count % inc1 == 0:
#     print('folder '+str(count)+': '+im_group+'--- %10.6f seconds ---' % (time.time() - time_start))
#     time_start = time.time()

    ####################################################################
#     if count % inc2 == 0 or count == len(groups2run)-1:
#         print('saved files: '+str(range_start)+'_'+str(range_end))
#         for param in return_param:
#             np.savetxt(dm_dir+'/'+param+'_'+str(range_start)+'_'+str(range_end)+'.txt',np.transpose(A._DM_dict[param]),fmt='%6.4E')
#     if count % inc2 == 0 or count == len(groups2run)-1:
#         print('saved files: '+str(range_start)+'_'+str(range_end))
#         for i in range(len(return_param)):
#             count_dm = 0
#             str_src = '_ls'
#             if 'pgd' in return_param[i]:
#                 str_dw = '_dw'+str(dw_cutoff)
#             for j in A._DM_dict[return_param[i]]:
#                 save_name = dm_dir+'/'+return_param[i]+'_'+dm_method[count_dm]+str_src+str_dw+'_'+str(range_start)+'_'+str(range_end)+'.txt'
#                 np.savetxt(save_name,np.transpose(A._DM_dict[return_param[i]][j]['output']),fmt='%6.4E')
#                 count_dm += 1

C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\src\edp\liq.py:653: RuntimeWarning: invalid value encountered in less
  b_0[dc<dc_cutoff] = 12.435
C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\src\edp\liq.py:656: RuntimeWarning: invalid value encountered in less
  b_ln_pgv[dc<dc_cutoff] = 0.301
C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\src\edp\liq.py:659: RuntimeWarning: invalid value encountered in less
  b_ln_vs30[dc<dc_cutoff] = -2.615
C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\src\edp\liq.py:662: RuntimeWarning: invalid value encountered in less
  b_precip[dc<dc_cutoff] = 5.556e-4
C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\src\edp\liq.py:665: RuntimeWarning: invalid value encountered in less
  b_sqrt_dc[dc<dc_cutoff] = -0.0287
C:\Users\barry\OneDrive - SlateGeotech\Fragility\OpenSRA\src\edp\liq.py:668: RuntimeWarning: invalid value encountered in less
  b_dr[dc<dc_cutoff] = 0.0666
C:\Users\barry\OneDrive - SlateGeotech\

10
hazus_2014_rr
[0. 0. 0. 0. 0. 0. 0. 0.]
 
8
hazus_2014_rr
[0.         0.         0.         0.         1.52590584 0.
 0.         0.        ]
 
62
hazus_2014_rr
[0.         0.         1.59610284 0.         0.25570261 0.
 0.         0.        ]
 
10
orourke_2020_rr
[0. 0. 0. 0. 0. 0. 0. 0.]
 
8
orourke_2020_rr
[0.         0.         0.         0.         6.12979815 0.
 0.         0.        ]
 
62
orourke_2020_rr
[0.         0.         5.13507009 0.         2.16308391 0.
 0.         0.        ]
 


In [ ]:
for i in range(len(return_param)):
    count_dm = 0
    if 'pgd' in return_param[i]:
        str_dw = '_dw'+str(dw_cutoff)
    for j in A._DM_dict[return_param[i]]:
        save_name = dm_dir+'/'+return_param[i]+'_'+dm_method[count_dm]+str_dw+'_'+str(range_start)+'_'+str(range_end)+'.txt'
        np.savetxt(save_name,np.transpose(A._DM_dict[return_param[i]][j]['output']),fmt='%6.4E')
        count_dm += 1

In [8]:
np.divide([2,2],[1,1])

array([2., 2.])

In [8]:
pgv = A._IM_dict['pgv'][5].toarray()
pga = A._IM_dict['pga'][5].toarray()
liq_susc = A._EDP_dict['liq_susc']['method1']['output']
pgd = A._EDP_dict['pgd_ls']['method1']['output'][5].toarray()
pliq = A._EDP_dict['p_liq']['method1']['output'][5].toarray()
print(A._rup_site_dict['site_lon'][ind_marc])
print(A._rup_site_dict['site_lat'][ind_marc])
print(dwWB[ind_marc])
print(liq_susc[ind_marc])
for i in rup_marc:
#     if pgv[i,ind_marc[0]] > 0:
    print(str(i)+': '+str(A._RV_dict['rup']['M'][i])+', '+str(A._RV_dict['rup']['rate'][i]))
#     print(pgv[i,ind_marc])
#     print(pga[i,ind_marc])

    print(pgd[i,ind_marc])
    print(pliq[i,ind_marc])

[-121.959102 -121.969138 -121.66069  -121.660565 -121.508865 -121.507891
 -121.269766 -121.26935 ]
[37.952594 37.944882 38.012081 38.020377 37.973593 37.983513 38.128762
 38.135931]
[0.342592  0.0553938 0.0276969 0.238258  0.0391693 0.607441  2.18174
 1.36391  ]
['moderate' 'moderate' 'very high' 'high' 'high' 'high' 'high' 'high']
10: 5.95459510870155, 0.00012348449023624983
[0. 0. 0. 0. 0. 0. 0. 0.]
[ 8.31851412  5.45768385 27.64271134 25.3532038  21.78835072 15.57975862
 22.33922345 26.66938283]
8: 6.639466161281467, 0.001702372352063113
[ 0.          0.          0.          0.         15.01344423  0.
  0.          0.        ]
[20.07374108 15.44435967 39.30648907 42.85350157 33.47331679 23.56669088
 29.95841069 38.89529935]
62: 7.618024588567435, 0.0001418795360809041
[0.         0.         8.61817159 0.         0.5733024  0.
 0.         0.        ]
[19.03417737 12.89887906 47.7780469  49.2104892  34.91726829 33.49198141
 48.49180242 51.75406691]


In [20]:
type(liq_susc[ind_marc][2])

numpy.str_

In [9]:
b = np.array([10,5])

In [13]:
a = np.empty(2,dtype='<U12')
a[b<10] = 'none'
a[b>=10] = 'very high'
a

array(['very high', 'none'], dtype='<U12')

In [ ]:

		a = np.empty(nSites,dtype='<U8')

In [ ]:
## Ground settlement
time_start = time.time()

flag_sample_exist = True
im_group = groups2run[0]

#####################################################################
## load GM predictions and create random variable
A.create_RV(im_tool, site_loc_file, im_dir, im_list, rup_meta_file, flag_clear_dict=True, 
            flag_sample_exist=flag_sample_exist, im_group=im_group, num_rups=num_rups)

#####################################################################
## get liq_susc from Zhu et al. (2017)
category = 'liq'
method = 'zhu_etal_2017'
return_param = ['liq_susc']
dc_cutoff = 20
A.get_EDP(category=category, method=method, return_param=return_param, 
          vs30=vs30, precip=precip, dc=dc, dr=dr, dw=dw, 
          wtd=wtd, dc_cutoff=dc_cutoff)

## get pgd_gs from HAZUS (FEMA, 2014)
category = 'gs'
method = 'hazus_2014_gs'
return_param = ['pgd_gs']
source_dict = ['_EDP_dict']
source_param = ['liq_susc']
source_method = ['zhu_etal_2017']
A.get_EDP(category=category, method=method, return_param=return_param, 
          source_dict=source_dict, source_param=source_param, source_method=source_method)

#####################################################################
dm_method = []
## get rr from ALA (2001)
category = 'rr'
method = 'orourke_2020_rr'
return_param = ['rr_pgd']
source_dict = ['_EDP_dict']
source_param = ['pgd_gs']
source_method = ['hazus_2014_gs']
pgd_label = 'pgd_gs'
pgd_cutoff = 0
dm_method.append(method[0:method.find('_')])
A.get_DM(category=category, method=method, return_param=return_param,
         source_dict=source_dict, source_param=source_param, source_method=source_method, nsamp=1,
         pgd_cutoff=pgd_cutoff, pgd_label=pgd_label)

####################################################################
print('saved file')
for i in range(len(return_param)):
	count_dm = 0
	str_src = '_gs'
	str_dw = None
	for j in A._DM_dict[return_param[i]]:
		save_name = dm_dir+'/'+return_param[i]+'_'+dm_method[count_dm]+str_src+'_all.txt'
		np.savetxt(save_name,np.transpose(A._DM_dict[return_param[i]][j]['output']),fmt='%6.4E')
		count_dm += 1

In [33]:
dim_rup = 100
dim_d = len(A._rup_site_dict['site_lon'])
src_list = [78,144,191,221,247,297,331,337,350,364]
count = 0
for src in src_list:
    seg_list = np.loadtxt(dir_intersect+'/src_'+str(src)+'.txt',dtype=int)
    for seg in seg_list:
        rows.append(count)
        cols.append(seg)
    count += 1
rows = np.asarray(rows)
cols = np.asarray(cols)
mat = sparse.coo_matrix((np.ones(len(rows)),(rows,cols)),shape=(dim_rup,dim_d))

In [17]:
M = A._RV_dict['rup']['M']

In [19]:
M_sparse = sparse.coo_matrix((M[mat.row],(mat.row,mat.col)),shape=mat.shape)

In [20]:
10**M_sparse

TypeError: unsupported operand type(s) for ** or pow(): 'int' and 'coo_matrix'

In [14]:
## Surface fault rupture
time_start = time.time()

dir_intersect = r'C:\Users\barry\Desktop\OpenSHAInterface\src_intersect'
multi = 0
range_start = 0+100*multi
range_end = range_start+1
flag_sample_exist = True

##
for im_group in groups2run[range_start:range_end]:
    #####################################################################
    count += 1
    path_sample = im_dir+'/'+im_group+'/'
    
    #####################################################################
    ## load GM predictions and create random variable
    A.create_RV(im_tool, site_loc_file, im_dir, im_list, rup_meta_file, flag_clear_dict=True, 
                flag_sample_exist=flag_sample_exist, im_group=im_group, num_rups=num_rups)

    #####################################################################
    ## set up for surface fault rupture
    rows = []
    cols = []
    dim_rup = len(A._RV_dict['rup']['src'])
    print(dim_rup)
    dim_d = len(A._rup_site_dict['site_lon'])
    count = 0
    for src in A._RV_dict['rup']['src']:
        try:
            seg_list = np.loadtxt(dir_intersect+'/src_'+str(src)+'.txt',dtype=int,ndmin=1)
        except Warning():
            pass
        for seg in seg_list:
            rows.append(count)
            cols.append(seg)
        count += 1
    rows = np.asarray(rows)
    cols = np.asarray(cols)
    mat = sparse.coo_matrix((np.ones(len(rows)),(rows,cols)),shape=(dim_rup,dim_d))
    
    #####################################################################
    ## get pgd_gs from HAZUS (FEMA, 2014)
    category = 'surf'
    method = 'wells_coppersmith_1994'
    return_param = ['pgd_surf']
    flag_M = True
    A.get_EDP(category=category, method=method, return_param=return_param, flag_M=flag_M, mat_seg2calc=mat)

    #####################################################################
    dm_method = []
    ## get rr from ALA (2001)
    category = 'rr'
    method = 'orourke_2020_rr'
    return_param = ['rr_pgd']
    source_dict = ['_EDP_dict']
    source_param = ['pgd_surf']
    source_method = ['wells_coppersmith_1994']
    pgd_label = 'pgd_surf'
    pgd_cutoff = 1
    dm_method.append(method[0:method.find('_')])
    A.get_DM(category=category, method=method, return_param=return_param,
             source_dict=source_dict, source_param=source_param, source_method=source_method, nsamp=1,
             pgd_cutoff=pgd_cutoff, pgd_label=pgd_label)

    #####################################################################
#     if count % inc1 == 0:
    print('folder '+str(count)+': '+im_group+'--- %10.6f seconds ---' % (time.time() - time_start))
    time_start = time.time()

#     ####################################################################
#     print('saved file')
#     for i in range(len(return_param)):
#         count_dm = 0
#         str_src = '_gs'
#         str_dw = None
#         for j in A._DM_dict[return_param[i]]:
#             save_name = dm_dir+'/'+return_param[i]+'_'+dm_method[count_dm]+str_src+'_all.txt'
#             np.savetxt(save_name,np.transpose(A._DM_dict[return_param[i]][j]['output']),fmt='%6.4E')
#             count_dm += 1

100


C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: loadtxt: Empty input file: "C:\Users\barry\Desktop\OpenSHAInterface\src_intersect/src_221.txt"
C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: loadtxt: Empty input file: "C:\Users\barry\Desktop\OpenSHAInterface\src_intersect/src_247.txt"
C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: loadtxt: Empty input file: "C:\Users\barry\Desktop\OpenSHAInterface\src_intersect/src_297.txt"
C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: loadtxt: Empty input file: "C:\Users\barry\Desktop\OpenSHAInterface\src_intersect/src_364.txt"
C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: loadtxt: Empty input file: "C:\Users\barry\Desktop\OpenSHAInterface\src_intersect/src_659.txt"
C:\Users\barry\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: UserWarning: loadtxt: Empty input file: "C:\Us

folder 100: 0_99---   0.974392 seconds ---


In [15]:
A._EDP_dict

{'pgd_surf': {'method1': {'method': 'wells_coppersmith_1994',
   'source_param': None,
   'source_method': None,
   'output': <100x123316 sparse matrix of type '<class 'numpy.float64'>'
   	with 297 stored elements in COOrdinate format>}}}